<a href="https://colab.research.google.com/github/BBripy207/finalproyect26/blob/main/proyecto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py


Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 395 (delta 92), reused 53 (delta 51), pack-reused 269
Receiving objects: 100% (395/395), 108.50 KiB | 1.90 MiB/s, done.
Resolving deltas: 100% (194/194), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.1 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!wget https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

--2023-11-12 21:54:31--  https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 201.98.60.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|201.98.60.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18045440 (17M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip’

datos_abiertos_covi 100%[===================>]  17.21M  12.9MB/s    in 1.3s    

2023-11-12 21:54:33 (12.9 MB/s) - ‘datos_abiertos_covid19.zip’ saved [18045440/18045440]



In [3]:
!unzip /content/datos_abiertos_covid19.zip

Archive:  /content/datos_abiertos_covid19.zip
  inflating: COVID19MEXICO.csv       


In [4]:
import numpy as np
import pandas as pd
data = pd.read_csv('COVID19MEXICO.csv',low_memory=False)
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [5]:
from re import X
df = data
df = df.drop(columns=['SECTOR', 'MUNICIPIO_RES', 'ID_REGISTRO', 'ORIGEN', 'ENTIDAD_UM', 'NACIONALIDAD', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'HABLA_LENGUA_INDIG', 'INDIGENA', 'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN'])
df.columns
df['FECHA_ACTUALIZACION'] = pd.to_datetime(df['FECHA_ACTUALIZACION'])
df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'])
df['FECHA_INGRESO'] = pd.to_datetime(df['FECHA_INGRESO'])
df['FECHA_DEF'] = pd.to_datetime(df['FECHA_DEF'],errors='coerce')
df
for col in ['TIPO_PACIENTE']:
  df[col]=df[col].astype(str)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in ['TIPO_PACIENTE','SEXO']:
  df[col]=le.fit_transform(df[col])

df['DIAS_DEFUNCION']=(df['FECHA_ACTUALIZACION']-df['FECHA_DEF']).dt.days
df['DIAS_hospital']=(df['FECHA_ACTUALIZACION']-df['FECHA_INGRESO']).dt.days
df['DIAS_SINTOMAS']=(df['FECHA_ACTUALIZACION']-df['FECHA_SINTOMAS']).dt.days
df['DIAS_DEFUNCION'] = df['DIAS_DEFUNCION'].apply(lambda x: 0 if x < 0 else x)
df['DIAS_DEFUNCION']
df = df.drop(columns=['FECHA_ACTUALIZACION','FECHA_SINTOMAS','FECHA_INGRESO','FECHA_DEF','DIAS_DEFUNCION'])
X=df.drop(columns=['CLASIFICACION_FINAL'])
y=df['CLASIFICACION_FINAL']


In [6]:
df.columns

Index(['SEXO', 'TIPO_PACIENTE', 'INTUBADO', 'NEUMONIA', 'EDAD', 'EMBARAZO',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'UCI', 'DIAS_hospital', 'DIAS_SINTOMAS'],
      dtype='object')

In [7]:
valores_a_eliminar = [3, 4]
filas_a_eliminar = df.loc[df['RESULTADO_LAB'].isin(valores_a_eliminar)]

# Muestra las filas seleccionadas
df = df.loc[~df['RESULTADO_LAB'].isin(valores_a_eliminar)]

df.head()

,SEXO,TIPO_PACIENTE,INTUBADO,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,...,TABAQUISMO,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,UCI,DIAS_hospital,DIAS_SINTOMAS
0,1,0,97,2,28,97,2,2,2,2,...,1,2,2,97,1,2,7,97,243,246
1,0,0,97,2,54,2,2,2,2,2,...,2,2,2,97,2,97,6,97,295,297
2,0,0,97,2,65,2,1,2,2,2,...,2,1,2,97,1,1,3,97,253,254
3,0,0,97,2,62,2,2,2,2,2,...,2,1,2,97,1,1,3,97,227,230
4,1,0,97,2,40,97,2,2,2,2,...,2,1,2,97,1,1,3,97,288,288


In [6]:
import pandas as pd

# Supongamos que tu DataFrame se llama df y la variable objetivo es 'CLASIFICACION_FINAL'
# Muestra el DataFrame original


# Calcular el umbral en función de un porcentaje del tamaño total del conjunto de datos
umbral_porcentaje = 0.01
umbral_instancias = 10000

# Identificar clases con pocas instancias (menos de umbral_instancias)
clases_pocas_instancias = df['CLASIFICACION_FINAL'].value_counts()[df['CLASIFICACION_FINAL'].value_counts() < umbral_instancias].index.tolist()
print(clases_pocas_instancias)

[1, 2]


In [8]:
clases_a_eliminar = [2, 4, 5]

# Filtrar el DataFrame para excluir las filas con las clases a eliminar
df = df[~df['CLASIFICACION_FINAL'].isin(clases_a_eliminar)]

# Muestra el DataFrame después de eliminar las clases
print("\nDataFrame después de eliminar las clases:")
print(df.head())


DataFrame después de eliminar las clases:
   SEXO  TIPO_PACIENTE  INTUBADO  NEUMONIA  EDAD  EMBARAZO  DIABETES  EPOC  \
0     1              0        97         2    28        97         2     2   
1     0              0        97         2    54         2         2     2   
2     0              0        97         2    65         2         1     2   
3     0              0        97         2    62         2         2     2   
4     1              0        97         2    40        97         2     2   

   ASMA  INMUSUPR  ...  TABAQUISMO  OTRO_CASO  TOMA_MUESTRA_LAB  \
0     2         2  ...           1          2                 2   
1     2         2  ...           2          2                 2   
2     2         2  ...           2          1                 2   
3     2         2  ...           2          1                 2   
4     2         2  ...           2          1                 2   

   RESULTADO_LAB  TOMA_MUESTRA_ANTIGENO  RESULTADO_ANTIGENO  \
0             97      

In [12]:
print(df['CLASIFICACION_FINAL'].value_counts())

7    688876
3    417387
6     57852
1      7650
Name: CLASIFICACION_FINAL, dtype: int64


In [9]:
from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf = RandomForestClassifier(random_state=42)
 #clf.fit(X_train,y_train)
 #y_predict=clf.predict(X_train)
 #print(classification_report(y_train,y_predict))

In [11]:
import cuml
from cuml.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf=RandomForestClassifier(random_state=42)
#
X_train=X_train.astype('float32')
y_train=y_train.astype('float32')

clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

print(classification_report(y_test,y_pred))

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       1.00      0.01      0.02      1673
           2       0.00      0.00      0.00         5
           3       1.00      1.00      1.00     84200
           4       0.00      0.00      0.00        35
           5       0.59      0.22      0.32       396
           6       0.94      0.97      0.96     12816
           7       0.99      1.00      0.99    138214

    accuracy                           0.99    237339
   macro avg       0.65      0.46      0.47    237339
weighted avg       0.99      0.99      0.99    237339



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
df.head()

,SEXO,TIPO_PACIENTE,INTUBADO,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,...,TABAQUISMO,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,UCI,DIAS_hospital,DIAS_SINTOMAS
0,1,0,97,2,28,97,2,2,2,2,...,1,2,2,97,1,2,7,97,243,246
1,0,0,97,2,54,2,2,2,2,2,...,2,2,2,97,2,97,6,97,295,297
2,0,0,97,2,65,2,1,2,2,2,...,2,1,2,97,1,1,3,97,253,254
3,0,0,97,2,62,2,2,2,2,2,...,2,1,2,97,1,1,3,97,227,230
4,1,0,97,2,40,97,2,2,2,2,...,2,1,2,97,1,1,3,97,288,288


In [20]:
df['CLASIFICACION_FINAL'].unique()

array([7, 6, 3, 1])

In [13]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [14]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10

In [16]:
import gradio as gr
import pandas as pd
import numpy as np
DESCRIPCION_CLAVES = """
CLAVE   DESCRIPCIÓN
1       SI
2       NO
97      NO APLICA
98      SE IGNORA
99      NO ESPECIFICADO
"""

def predict(sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
             diabetes, epoc, asma, inmusupr,
            hipertension, otra_com, cardiovascular, obesidad,
            renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
            resultado_lab, toma_muestra_antigeno, resultado_antigeno,
           uci, DIAS_hospital, DIAS_SINTOMAS):


    # Crea un DataFrame con los valores de entrada
    input_data = pd.DataFrame([[
        sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
       diabetes, epoc, asma, inmusupr,
        hipertension, otra_com, cardiovascular, obesidad,
        renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
        resultado_lab, toma_muestra_antigeno, resultado_antigeno, uci, DIAS_hospital, DIAS_SINTOMAS
        ]], columns=[
        'SEXO', 'TIPO_PACIENTE', 'INTUBADO', 'NEUMONIA', 'EDAD', 'EMBARAZO','DIABETES', 'EPOC',
        'ASMA', 'INMUSUPR','HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD','RENAL_CRONICA',
        'TABAQUISMO', 'OTRO_CASO', 'TOMA_MUESTRA_LAB','RESULTADO_LAB', 'TOMA_MUESTRA_ANTIGENO',
        'RESULTADO_ANTIGENO','UCI', 'DIAS_hospital', 'DIAS_SINTOMAS'])

    prediction = clf.predict(input_data)
    return prediction




gr.Row

inputs = [
    gr.Radio(choices=[0, 1], label='TU Sexo'),
    gr.Radio(choices=[0, 1], label='Cual es el Tipo de Paciente'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Esta Intubado?'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Neumonía'),
    gr.Slider(minimum=0, maximum=120, value=30, label='Que edad Edad tiene?'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Embarazo'),
    gr.Radio(choices=[1, 2, 98], label='Diabetes'),
    gr.Radio(choices=[1, 2, 98], label='EPOC'),
    gr.Radio(choices=[1, 2, 98], label='Asma'),
    gr.Radio(choices=[1, 2, 98], label='Inmunosupresión'),
    gr.Radio(choices=[1, 2, 98], label='Hipertensión'),
    gr.Radio(choices=[1, 2, 98], label='Otra Comorbilidad'),
    gr.Radio(choices=[1, 2, 98], label='Cardiovascular'),
    gr.Radio(choices=[1, 2, 98], label='Obesidad'),
    gr.Radio(choices=[1, 2, 98], label='Enfermedad Renal Crónica'),
    gr.Radio(choices=[1, 2, 98], label='Tabaquismo'),
    gr.Radio(choices=[1, 2, 99], label='Contacto con otro caso'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de laboratorio'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de laboratorio'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='UCI'),
    gr.Number(label='DIAS_hospital',value=0),
    gr.Number(label='DIAS_SINTOMAS',value=0),
]

outputs = gr.Textbox(label="Tu  Predicción es ")

# Crear la interfaz de Gradio
demo = gr.Interface(fn=predict, inputs=inputs, outputs=outputs)


# Ejecutar la aplicación web
if __name__ == "__main__":
    demo.launch(show_api=False,debug=True)

ImportError: ignored